# Тематизация жалоб без эмбеддингов: LDA и LSI+KMeans

Этот ноутбук содержит полностью рабочие пайплайны без использования эмбеддингов HuggingFace:
- LDA (CountVectorizer → LatentDirichletAllocation)
- LSI (TF‑IDF → TruncatedSVD) + KMeans

Включено:
- Очистка текста (минимальная, с заменой PII)
- Обучение, предсказание, извлечение ключевых слов для тем/кластеров
- Метрики: LDA перплексия, silhouette для KMeans
- Сохранение/загрузка моделей
- Демонстрация на примерах

Подойдёт как стартовый шаблон для кластеризации обращений клиентов.


In [ ]:
# При необходимости раскомментируйте для установки
# %pip install scikit-learn pandas numpy joblib razdel nltk

import re
import os
import json
import joblib
import numpy as np
import pandas as pd

from typing import List, Tuple, Dict, Optional

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Токены, которые не должны попадать в названия тем/топ-термины
BLOCKED_TERMS = {
    "amount", "card", "date", "email",
    "pii_amount", "pii_card", "pii_date", "pii_email",
    # базовые доменные токены, чтобы не появлялись в названиях
    "банк","банка","банку","банком","банке","банков","банкам","банках","банками",
    "клиент","клиента","клиенту","клиентом","клиенты","клиентам","клиентов","клиентами","клиентах",
    # вежливости/мета
    "добрый","день","здравствуйте","пожалуйста","коллеги","уважаемый","уважаемая","уважаемые",
    "просьба","просит","обращение","ответ",
}

# Русские стоп-слова (жёстко зашитый список без NLTK) + доменные слова
RU_STOPWORDS = {
    "и","в","во","не","что","он","на","я","с","со","как","а","то","все","она","так","его","но","да","ты","к","у","же","вы","за","бы","по","только","ее","мне","было","вот","от","меня","еще","нет","о","из","ему","теперь","когда","даже","ну","вдруг","ли","если","уже","или","ни","быть","был","него","до","вас","нибудь","опять","уж","вам","ведь","там","потом","себя","ничего","ей","может","они","тут","где","есть","надо","ней","для","мы","тебя","их","чем","была","сам","чтоб","без","будто","чего","раз","тоже","себе","под","будет","ж","тогда","кто","этот","того","потому","этого","какой","совсем","ним","здесь","этом","один","почти","мой","тем","чтобы","нее","сейчас","были","куда","зачем","всех","никогда","можно","при","наконец","два","об","другой","хоть","после","над","больше","тот","через","эти","нас","про","всего","них","какая","много","разве","три","эту","моя","впрочем","хорошо","свою","этой","перед","иногда","лучше","чуть","том","нельзя","такой","им","более","всегда","конечно","всю","между"
}

EXTRA_STOPWORDS = {
    # вежливости/мета
    "добрый","день","здравствуйте","пожалуйста","коллеги","уважаемый","уважаемая","уважаемые",
    # банк/клиент (частые формы)
    "клиент","клиента","клиенту","клиентом","клиенты","клиентам","клиентов","клиентами","клиентах",
    "банк","банка","банку","банком","банке","банков","банкам","банках","банками",
    # мета-лексема
    "просьба","просит","обращение","ответ"
}

# Один объединённый список стоп-слов (list), совместимый со sklearn
STOP_WORDS = sorted(RU_STOPWORDS | EXTRA_STOPWORDS)



In [ ]:
# Загрузка данных
df = pd.read_csv('output - 2025-10-29T194929.261.csv', encoding='utf-16', sep='(t')
# Фильтрация по типу заявления (поле гарантированно есть)
allowed_types = {"Жалоба", "Претензия", "Предложение"}
df = df[df['Тип заявления'].astype(str).isin(allowed_types)].copy()
# Пересобираем тексты после фильтрации
texts = df['Описание претензии'].astype(str).fillna("").tolist()


In [ ]:
# Очистка текста

PII_EMAIL = re.compile(r"\S+@\S+")
PII_CARD = re.compile(r"\b\d{12,16}\b")
PII_DATE = re.compile(r"\d{1,2}[./-]\d{1,2}[./-]\d{2,4}")
PII_AMOUNT = re.compile(r"\d+[ ,\.]?\d*\s?(₽|р\.?|rub|\$|usd|eur)?", re.IGNORECASE)
PII_PHONE = re.compile(r"(\+7|8)[\s\-\(\)]?\d[\d\s\-\(\)]{8,14}")
PII_PASSPORT = re.compile(r"(паспорт|серия|сер\.)[^\n\r]*?(\d{2,4})[^\n\r]*?(номер|№|n)?[^\n\r]*?(\d{6})", re.IGNORECASE)
PII_DOB_FIELD = re.compile(r"(дата\s+рождения\s*:?)[^\n\r]*", re.IGNORECASE)
PII_FIO_FIELD = re.compile(r"(фамилия[,\s]*имя[,\s]*отчество\s*:?)[^\n\r]*", re.IGNORECASE)
PII_ADDRESS_FIELD = re.compile(r"(адрес(\s+постоянной\s+регистрации)?\s*:?)[^\n\r]*", re.IGNORECASE)
PII_PHONE_FIELD = re.compile(r"(контактный\s+телефон\s*:?)[^\n\r]*", re.IGNORECASE)

# Удаляем URL и строки вида "Адрес отзыва: ..."
RE_URL = re.compile(r"https?://\S+", re.IGNORECASE)
RE_REVIEW_ADDR_LINE = re.compile(r"адрес\s+отзыва:.*", re.IGNORECASE)

# Удаляем почтовые заголовки и служебные поля
RE_MAIL_HEADERS = re.compile(r"^(кому|копия|тема|получатель|от|cc|bcc|файл\s+обращения|контакт\-?центр)\b.*", re.IGNORECASE | re.MULTILINE)
# Удаляем служебные мета-уведомления
RE_META_NOTICES = re.compile(r"^(пользователем\s+.*?были\s+запрошены|в\s+\"?народный\s+рейтинг\"?.*добавлен.*отзыв|заголовок:|оценка:|автор:|дата:|пункт\s+обслуживания:|дополнительная\s+информация).*", re.IGNORECASE | re.MULTILINE)

# Приветствия/шапки
RE_GREETINGS = re.compile(r"^(д\.д\.|дд\!?|добрый\s+день\!?)[\s,]*", re.IGNORECASE)
RE_META_PHRASES = re.compile(r"\b(со\s+слов\s+клиента|клиент\s+пишет)\b[:,\s]*", re.IGNORECASE)

# Входящий документ (мета-записи)
RE_INCOMING_DOC_LINE = re.compile(r"^входящий\s+документ\s*(№|n|no)?\s*\d+\s*от\s*<date>.*$", re.IGNORECASE | re.MULTILINE)
RE_INCOMING_DOC_RAW = re.compile(r"^входящий\s+документ\b.*", re.IGNORECASE)

RE_MULTISPACE = re.compile(r"\s{2,}")


def clean_text(text: str) -> str:
    """Расширенная очистка: удаление служебных строк, URL, замена PII/телефонов/паспортов, упрощение шапок."""
    t = str(text).lower()
    # удалить целиком строки "Адрес отзыва: ...", почтовые заголовки и мета-уведомления
    t = RE_REVIEW_ADDR_LINE.sub(" ", t)
    t = RE_MAIL_HEADERS.sub(" ", t)
    t = RE_META_NOTICES.sub(" ", t)
    # удалить URL
    t = RE_URL.sub(" ", t)
    # привести/срезать шапки типа ДД/добрый день и мета-фразы
    t = RE_GREETINGS.sub(" ", t)
    t = RE_META_PHRASES.sub(" ", t)
    # PII замены
    t = PII_EMAIL.sub("<email>", t)
    t = PII_CARD.sub("<card>", t)
    t = PII_DATE.sub("<date>", t)
    t = PII_AMOUNT.sub("<amount>", t)
    t = PII_PHONE.sub("<phone>", t)
    # Срез полей с персональными данными (оставляем маркеры, убираем значения)
    t = PII_DOB_FIELD.sub("дата рождения: <date>", t)
    t = PII_FIO_FIELD.sub("фио: <name>", t)
    t = PII_ADDRESS_FIELD.sub("адрес: <address>", t)
    t = PII_PHONE_FIELD.sub("контактный телефон: <phone>", t)
    # Паспорт (серия/номер)
    t = PII_PASSPORT.sub(" <passport> ", t)
    # нормализация пробелов
    t = RE_MULTISPACE.sub(" ", t).strip()
    return t


def clean_corpus(texts: List[str]) -> List[str]:
    return [clean_text(t) for t in texts]


def should_drop_text(raw_text: str) -> bool:
    """Определить мета-записи без содержимого (например, только "Входящий документ № ... от ...")."""
    t = str(raw_text).lower().strip()
    if not t:
        return True
    if RE_INCOMING_DOC_RAW.match(t):
        # уберем числа/даты/знаки и проверим остаток
        tmp = PII_DATE.sub(" ", t)
        tmp = re.sub(r"[№nno\-:;.,\s\d]+", " ", tmp)
        tmp = RE_MULTISPACE.sub(" ", tmp).strip()
        return len(tmp) < 8  # почти пустая мета-строка
    return False



In [ ]:
# Фильтрация мета-записей после очистки (опционально)
# Удалим строки, которые состоят из служебного "входящий документ ..." без сути
cleaned_texts = clean_corpus(texts)
mask_keep = [not should_drop_text(t) for t in texts]
texts = [t for t, keep in zip(cleaned_texts, mask_keep) if keep]
print(f"Оставлено документов: {len(texts)}")



In [ ]:
# Пайплайн LDA

class LdaTopicModel:
    def __init__(
        self,
        n_topics: int = 10,
        max_df: float = 0.95,
        min_df: int = 5,
        ngram_range: Tuple[int, int] = (1, 2),
        random_state: int = 42,
    ) -> None:
        self.n_topics = n_topics
        self.vectorizer = CountVectorizer(
            stop_words=STOP_WORDS,
            max_df=max_df,
            min_df=min_df,
            ngram_range=ngram_range,
        )
        self.lda = LatentDirichletAllocation(
            n_components=n_topics,
            learning_method="batch",
            max_iter=20,
            random_state=random_state,
            n_jobs=-1,
        )
        self.is_fitted = False

    def fit(self, texts: List[str]) -> "LdaTopicModel":
        cleaned = clean_corpus(texts)
        X = self.vectorizer.fit_transform(cleaned)
        self.doc_topic = self.lda.fit_transform(X)
        self.is_fitted = True
        return self

    def transform(self, texts: List[str]) -> np.ndarray:
        assert self.is_fitted, "Сначала вызовите fit()"
        cleaned = clean_corpus(texts)
        X = self.vectorizer.transform(cleaned)
        return self.lda.transform(X)

    def predict(self, texts: List[str]) -> Tuple[np.ndarray, np.ndarray]:
        dist = self.transform(texts)
        return dist.argmax(axis=1), dist.max(axis=1)

    def get_topic_top_words(self, top_n: int = 10) -> Dict[int, List[str]]:
        assert self.is_fitted, "Сначала вызовите fit()"
        feature_names = self.vectorizer.get_feature_names_out()
        topics: Dict[int, List[str]] = {}
        for i, comp in enumerate(self.lda.components_):
            top_idx = comp.argsort()[::-1]
            words: List[str] = []
            for j in top_idx:
                w = feature_names[j]
                if w in BLOCKED_TERMS:
                    continue
                words.append(w)
                if len(words) >= top_n:
                    break
            topics[i] = words
        return topics

    def to_dataframe(self, texts: List[str]) -> pd.DataFrame:
        assert self.is_fitted, "Сначала вызовите fit()"
        preds, conf = self.predict(texts)
        return pd.DataFrame({"text": texts, "topic_id": preds, "topic_confidence": conf})

    def perplexity(self, texts: Optional[List[str]] = None) -> float:
        assert self.is_fitted, "Сначала вызовите fit()"
        if texts is None:
            texts = self._last_fit_corpus if hasattr(self, "_last_fit_corpus") else None
        cleaned = clean_corpus(texts) if texts is not None else None
        X = self.vectorizer.transform(cleaned) if cleaned is not None else None
        if X is None:
            raise ValueError("Передайте texts для оценки перплексии")
        return float(self.lda.perplexity(X))

    def save(self, path: str) -> None:
        os.makedirs(path, exist_ok=True)
        joblib.dump(self.vectorizer, os.path.join(path, "vectorizer.joblib"))
        joblib.dump(self.lda, os.path.join(path, "lda.joblib"))
        meta = {"n_topics": self.n_topics}
        with open(os.path.join(path, "meta.json"), "w", encoding="utf-8") as f:
            json.dump(meta, f, ensure_ascii=False, indent=2)

    @classmethod
    def load(cls, path: str) -> "LdaTopicModel":
        with open(os.path.join(path, "meta.json"), "r", encoding="utf-8") as f:
            meta = json.load(f)
        obj = cls(n_topics=meta["n_topics"])
        obj.vectorizer = joblib.load(os.path.join(path, "vectorizer.joblib"))
        obj.lda = joblib.load(os.path.join(path, "lda.joblib"))
        obj.is_fitted = True
        return obj



In [ ]:
# Пайплайн LSI (TF-IDF + SVD) + KMeans

from collections import defaultdict

class LsiKMeansTopicModel:
    def __init__(
        self,
        n_components: int = 100,
        n_clusters: int = 10,
        max_df: float = 0.95,
        min_df: int = 5,
        ngram_range: Tuple[int, int] = (1, 2),
        random_state: int = 42,
    ) -> None:
        self.n_components = n_components
        self.n_clusters = n_clusters
        self.tfidf = TfidfVectorizer(
            stop_words=STOP_WORDS,
            max_df=max_df,
            min_df=min_df,
            ngram_range=ngram_range,
        )
        self.svd = TruncatedSVD(n_components=n_components, random_state=random_state)
        self.lsi = make_pipeline(self.svd, Normalizer(copy=False))
        self.kmeans = KMeans(n_clusters=n_clusters, n_init="auto", random_state=random_state)
        self.is_fitted = False

    def fit(self, texts: List[str]) -> "LsiKMeansTopicModel":
        cleaned = clean_corpus(texts)
        X_tfidf = self.tfidf.fit_transform(cleaned)
        X_lsi = self.lsi.fit_transform(X_tfidf)
        self.labels_ = self.kmeans.fit_predict(X_lsi)
        self.is_fitted = True
        return self

    def transform(self, texts: List[str]) -> np.ndarray:
        assert self.is_fitted, "Сначала вызовите fit()"
        cleaned = clean_corpus(texts)
        X_tfidf = self.tfidf.transform(cleaned)
        return self.lsi.transform(X_tfidf)

    def predict(self, texts: List[str]) -> np.ndarray:
        X_lsi = self.transform(texts)
        return self.kmeans.predict(X_lsi)

    def to_dataframe(self, texts: List[str]) -> pd.DataFrame:
        assert self.is_fitted, "Сначала вызовите fit()"
        labels = self.predict(texts)
        return pd.DataFrame({"text": texts, "topic_id": labels})

    def silhouette(self, texts: List[str]) -> float:
        assert self.is_fitted, "Сначала вызовите fit()"
        X_lsi = self.transform(texts)
        return float(silhouette_score(X_lsi, self.kmeans.predict(X_lsi)))

    def get_cluster_terms(self, texts: List[str], top_n: int = 10) -> Dict[int, List[str]]:
        """c-TF-IDF по агрегированным документам кластеров.
        Передайте исходные тексты корпуса (те же, на которых обучали).
        """
        assert self.is_fitted, "Сначала вызовите fit()"
        cleaned = clean_corpus(texts)
        # Аггрегируем по кластерам
        cluster_docs: Dict[int, List[str]] = defaultdict(list)
        for t, lab in zip(cleaned, self.labels_):
            cluster_docs[lab].append(t)
        agg_texts = [" ".join(cluster_docs[i]) if i in cluster_docs else "" for i in range(self.n_clusters)]
        # Считаем c-TF-IDF
        ctfidf = TfidfVectorizer(stop_words=STOP_WORDS, ngram_range=(1, 2), min_df=2)
        C = ctfidf.fit_transform(agg_texts)
        feat = ctfidf.get_feature_names_out()
        terms: Dict[int, List[str]] = {}
        for i in range(self.n_clusters):
            row = C[i]
            scores = row.toarray().ravel()
            order = scores.argsort()[::-1]
            words: List[str] = []
            for j in order:
                w = str(feat[j])
                if w in BLOCKED_TERMS:
                    continue
                words.append(w)
                if len(words) >= top_n:
                    break
            terms[i] = words
        return terms

    def save(self, path: str) -> None:
        os.makedirs(path, exist_ok=True)
        joblib.dump(self.tfidf, os.path.join(path, "tfidf.joblib"))
        joblib.dump(self.svd, os.path.join(path, "svd.joblib"))
        joblib.dump(self.lsi, os.path.join(path, "lsi_pipeline.joblib"))
        joblib.dump(self.kmeans, os.path.join(path, "kmeans.joblib"))
        meta = {"n_components": self.n_components, "n_clusters": self.n_clusters}
        with open(os.path.join(path, "meta.json"), "w", encoding="utf-8") as f:
            json.dump(meta, f, ensure_ascii=False, indent=2)

    @classmethod
    def load(cls, path: str) -> "LsiKMeansTopicModel":
        with open(os.path.join(path, "meta.json"), "r", encoding="utf-8") as f:
            meta = json.load(f)
        obj = cls(n_components=meta["n_components"], n_clusters=meta["n_clusters"])
        obj.tfidf = joblib.load(os.path.join(path, "tfidf.joblib"))
        obj.svd = joblib.load(os.path.join(path, "svd.joblib"))
        obj.lsi = joblib.load(os.path.join(path, "lsi_pipeline.joblib"))
        obj.kmeans = joblib.load(os.path.join(path, "kmeans.joblib"))
        obj.is_fitted = True
        return obj



In [ ]:
# Подбор числа тем/кластеров

def lda_try_topics(texts: List[str], topics_list: List[int]) -> pd.DataFrame:
    """Перебор числа тем для LDA и оценка перплексии (ниже — лучше)."""
    cleaned = clean_corpus(texts)
    cv = CountVectorizer(stop_words=STOP_WORDS, max_df=0.95, min_df=5, ngram_range=(1, 2))
    X = cv.fit_transform(cleaned)
    rows = []
    for k in topics_list:
        lda = LatentDirichletAllocation(n_components=k, learning_method="batch", max_iter=15, random_state=42, n_jobs=-1)
        lda.fit(X)
        perp = float(lda.perplexity(X))
        rows.append({"n_topics": k, "perplexity": perp})
    return pd.DataFrame(rows).sort_values("n_topics")


def lsi_kmeans_try_k(texts: List[str], k_list: List[int], n_components: int = 100) -> pd.DataFrame:
    """Перебор k для KMeans на LSI-признаках и оценка silhouette (выше — лучше)."""
    cleaned = clean_corpus(texts)
    tfidf = TfidfVectorizer(stop_words=STOP_WORDS, max_df=0.95, min_df=5, ngram_range=(1, 2))
    X_tfidf = tfidf.fit_transform(cleaned)
    lsi = make_pipeline(TruncatedSVD(n_components=n_components, random_state=42), Normalizer(copy=False))
    X_lsi = lsi.fit_transform(X_tfidf)
    rows = []
    for k in k_list:
        km = KMeans(n_clusters=k, n_init="auto", random_state=42)
        labels = km.fit_predict(X_lsi)
        sil = float(silhouette_score(X_lsi, labels))
        rows.append({"k": k, "silhouette": sil})
    return pd.DataFrame(rows).sort_values("k")



In [ ]:
# Демонстрация на примерах

example_texts = texts[50]

# 1) LDA
lda_model = LdaTopicModel(n_topics=10)
lda_model.fit(example_texts)
lda_topics_df = lda_model.to_dataframe(example_texts)
print("Топ-слова по темам (LDA):")
print(pd.DataFrame.from_dict(lda_model.get_topic_top_words(top_n=8), orient="index"))

display(lda_topics_df)

# 2) LSI+KMeans
lsi_model = LsiKMeansTopicModel(n_components=100, n_clusters=10)
lsi_model.fit(example_texts)
lsi_topics_df = lsi_model.to_dataframe(example_texts)
print("\nТоп-термины по кластерам (LSI+KMeans):")
print(pd.DataFrame.from_dict(lsi_model.get_cluster_terms(example_texts, top_n=8), orient="index"))

display(lsi_topics_df)



In [ ]:
# Применение к новым жалобам + сохранение/загрузка

new_texts = [
    "Прошу вернуть годовую комиссию, не было уведомления о списании",
    "Перевод на казахстанскую карту не дошёл, сделайте розыск",
]

lda_pred_ids, lda_pred_conf = lda_model.predict(new_texts)
print("LDA предсказания:")
print(pd.DataFrame({"text": new_texts, "topic_id": lda_pred_ids, "confidence": lda_pred_conf}))

lsi_pred_ids = lsi_model.predict(new_texts)
print("\nLSI+KMeans предсказания:")
print(pd.DataFrame({"text": new_texts, "topic_id": lsi_pred_ids}))

# Сохранение
lda_model.save("models/lda_model")
lsi_model.save("models/lsi_kmeans_model")

# Загрузка
lda_loaded = LdaTopicModel.load("models/lda_model")
lsi_loaded = LsiKMeansTopicModel.load("models/lsi_kmeans_model")

_ = lda_loaded.predict(["Возврат страховки по кредиту не произведён"])  # smoke-test
_ = lsi_loaded.predict(["Карта заблокирована без предупреждения"])  # smoke-test



## Пост-обработка: мэппинг тем в бизнес‑рубрики

- Создаем словарь соответствия `исходная_тема → бизнес‑тема`.
- LDA: суммируем вероятности по темам, входящим в одну бизнес‑тему, и берём максимум.
- Пересчитываем ключевые термины по финальным бизнес‑темам (c‑TF‑IDF).
- Анализируем размер финальных тем (шт. и % от корпуса).


In [ ]:
from collections import Counter

def apply_lda_mapping(lda_model: LdaTopicModel, texts: List[str], topic_to_business: Dict[int, str]) -> pd.DataFrame:
    """Возвращает финальную бизнес-метку и уверенность (сумма вероятностей) по документу."""
    dist = lda_model.transform(texts)
    final_labels: List[str] = []
    final_conf: List[float] = []
    for row in dist:
        agg: Dict[str, float] = {}
        for topic_id, p in enumerate(row):
            label = topic_to_business.get(topic_id, f"topic_{topic_id}")
            agg[label] = agg.get(label, 0.0) + float(p)
        best_label, best_prob = max(agg.items(), key=lambda x: x[1])
        final_labels.append(best_label)
        final_conf.append(best_prob)
    return pd.DataFrame({"text": texts, "final_label": final_labels, "final_conf": final_conf})


def apply_kmeans_mapping(lsi_model: LsiKMeansTopicModel, texts: List[str], cluster_to_business: Dict[int, str]) -> pd.DataFrame:
    labels = lsi_model.predict(texts)
    final_labels = [cluster_to_business.get(int(l), f"cluster_{int(l)}") for l in labels]
    return pd.DataFrame({"text": texts, "final_label": final_labels})


def compute_business_terms(texts: List[str], labels: List[str], top_n: int = 10) -> Dict[str, List[str]]:
    """Считает ключевые термины для финальных бизнес‑тем (c‑TF‑IDF)."""
    cleaned = clean_corpus(texts)
    groups: Dict[str, List[str]] = {}
    for t, lab in zip(cleaned, labels):
        groups.setdefault(lab, []).append(t)
    business_names = sorted(groups.keys())
    agg_texts = [" ".join(groups[name]) for name in business_names]
    vec = TfidfVectorizer(stop_words=STOP_WORDS, ngram_range=(1, 2), min_df=2)
    M = vec.fit_transform(agg_texts)
    feat = vec.get_feature_names_out()
    terms: Dict[str, List[str]] = {}
    for idx, name in enumerate(business_names):
        row = M[idx]
        scores = row.toarray().ravel()
        order = scores.argsort()[::-1]
        words: List[str] = []
        for j in order:
            w = str(feat[j])
            if w in BLOCKED_TERMS:
                continue
            words.append(w)
            if len(words) >= top_n:
                break
        terms[name] = words
    return terms


def business_size_report(labels: List[str]) -> pd.DataFrame:
    cnt = Counter(labels)
    total = sum(cnt.values())
    rows = [{"business_label": k, "count": v, "share": v / total} for k, v in sorted(cnt.items(), key=lambda x: -x[1])]
    return pd.DataFrame(rows)

# Пример использования (заполните mapping под ваши рубрики)
# Мэппинг для LDA: исходная_тема -> бизнес‑тема
lda_mapping_example = {i: f"Тема_{i}" for i in range(lda_model.lda.n_components)}
lda_final_df = apply_lda_mapping(lda_model, example_texts, lda_mapping_example)
lda_terms_final = compute_business_terms(example_texts, lda_final_df["final_label"].tolist(), top_n=8)
print("Финальные бизнес‑темы (LDA):")
print(business_size_report(lda_final_df["final_label"].tolist()))
print(pd.DataFrame.from_dict(lda_terms_final, orient="index"))

# Мэппинг для LSI+KMeans: кластер -> бизнес‑тема
kmeans_mapping_example = {i: f"Тема_{i}" for i in range(lsi_model.n_clusters)}
kmeans_final_df = apply_kmeans_mapping(lsi_model, example_texts, kmeans_mapping_example)
kmeans_terms_final = compute_business_terms(example_texts, kmeans_final_df["final_label"].tolist(), top_n=8)
print("\nФинальные бизнес‑темы (LSI+KMeans):")
print(business_size_report(kmeans_final_df["final_label"].tolist()))
print(pd.DataFrame.from_dict(kmeans_terms_final, orient="index"))



## Автонаименование тем из n‑грамм (1–3 слов)

Алгоритм: считаем c‑TF‑IDF по агрегатам тем, генерируем кандидатов (уни/би/триграммы), бустим фразы длиннее, фильтруем служебные и дубликаты‑включения, выбираем 1–2 лучших названий динамической длины.


In [ ]:
from typing import Iterable

def _is_blocked_phrase(phrase: str) -> bool:
    # блокируем, если вся фраза — плейсхолдер или если содержит чисто служебные токены
    tokens = phrase.split()
    if phrase in BLOCKED_TERMS:
        return True
    if any(tok in BLOCKED_TERMS for tok in tokens):
        return True
    return False


def generate_labels_for_groups(
    groups: Dict[int, List[str]],
    ngram_range: Tuple[int, int] = (1, 3),
    min_df: int = 2,
    top_n_candidates: int = 50,
    phrase_boost: Tuple[float, float, float] = (1.0, 1.1, 1.2),
    inclusion_margin: float = 0.15,
    final_top_k: int = 1,
) -> Dict[int, List[str]]:
    """
    Генерирует динамические названия тем по агрегированным группам документов.
    - phrase_boost: множители для уни/би/три-gram.
    - inclusion_margin: если более длинная фраза включает короткую и их скора близки (<15%), оставляем длинную.
    - final_top_k: сколько ярлыков вернуть на тему.
    """
    # агрегация
    group_ids = sorted(groups.keys())
    agg_texts = [" ".join(groups[i]) for i in group_ids]

    vec = TfidfVectorizer(stop_words=STOP_WORDS, ngram_range=ngram_range, min_df=min_df)
    M = vec.fit_transform(agg_texts)
    feat = vec.get_feature_names_out()

    def gram_len(term: str) -> int:
        return len(term.split())

    labels: Dict[int, List[str]] = {}
    for row_idx, gid in enumerate(group_ids):
        scores = M[row_idx].toarray().ravel()
        order = scores.argsort()[::-1]
        candidates: List[Tuple[str, float]] = []
        for j in order[: top_n_candidates * 3]:  # с запасом
            term = str(feat[j])
            if _is_blocked_phrase(term):
                continue
            gl = gram_len(term)
            if gl == 1:
                adj = scores[j] * phrase_boost[0]
            elif gl == 2:
                adj = scores[j] * phrase_boost[1]
            else:
                adj = scores[j] * phrase_boost[2]
            candidates.append((term, adj))
            if len(candidates) >= top_n_candidates:
                break
        # устранение включений: предпочитаем более длинные, если близкие по скору
        final: List[Tuple[str, float]] = []
        for term, sc in candidates:
            drop = False
            # если уже взята более длинная фраза, содержащая текущую, и её скор не существенно хуже
            for k, (t2, sc2) in enumerate(list(final)):
                if t2 in term and sc >= sc2 * (1 - inclusion_margin):
                    # текущая длиннее и не хуже — заменим
                    final[k] = (term, sc)
                    drop = True
                    break
                if term in t2 and sc2 >= sc * (1 - inclusion_margin):
                    # уже есть более длинная похожая — отбрасываем
                    drop = True
                    break
            if not drop:
                final.append((term, sc))
        final_sorted = [t for t, _ in sorted(final, key=lambda x: x[1], reverse=True)]
        labels[gid] = final_sorted[:final_top_k]
    return labels


def generate_lda_labels(
    lda_model: LdaTopicModel,
    texts: List[str],
    final_top_k: int = 1,
    ngram_range: Tuple[int, int] = (1, 3),
    min_df: int = 2,
) -> Dict[int, List[str]]:
    # назначаем документ в тему по argmax
    dist = lda_model.transform(texts)
    assigned = dist.argmax(axis=1)
    groups: Dict[int, List[str]] = {}
    for t, lab in zip(texts, assigned):
        groups.setdefault(int(lab), []).append(t)
    return generate_labels_for_groups(groups, ngram_range=ngram_range, min_df=min_df, final_top_k=final_top_k)


def generate_kmeans_labels(
    lsi_model: LsiKMeansTopicModel,
    texts: List[str],
    final_top_k: int = 1,
    ngram_range: Tuple[int, int] = (1, 3),
    min_df: int = 2,
) -> Dict[int, List[str]]:
    labels = lsi_model.predict(texts)
    groups: Dict[int, List[str]] = {}
    for t, lab in zip(texts, labels):
        groups.setdefault(int(lab), []).append(t)
    return generate_labels_for_groups(groups, ngram_range=ngram_range, min_df=min_df, final_top_k=final_top_k)

# Пример: автоназвания на примерах
lda_auto = generate_lda_labels(lda_model, example_texts, final_top_k=1, ngram_range=(1,3), min_df=1)
print("LDA автоназвания тем:", lda_auto)

kmeans_auto = generate_kmeans_labels(lsi_model, example_texts, final_top_k=1, ngram_range=(1,3), min_df=1)
print("KMeans автоназвания тем:", kmeans_auto)



### LDA: полный запуск (обучение → топ-термины → автоназвания → таблица)



In [ ]:
# Обучение LDA
lda_model = LdaTopicModel(n_topics=12, max_df=1.0, min_df=2, ngram_range=(1,2))
lda_model.fit(texts)

# Топ-термины тем
lda_top = pd.DataFrame.from_dict(lda_model.get_topic_top_words(top_n=10), orient="index")
display(lda_top)

# Автоназвания тем из n-грамм (1–3)
lda_auto = generate_lda_labels(lda_model, texts, final_top_k=1, ngram_range=(1,3), min_df=2)

# Таблица документов с темой и ярлыком
df_lda = lda_model.to_dataframe(texts)
df_lda["auto_label"] = df_lda["topic_id"].map({tid: (labs[0] if labs else "") for tid, labs in lda_auto.items()})
display(df_lda.head())



### LSI+KMeans: полный запуск (обучение → термины → автоназвания → таблица)



In [ ]:
# Обучение LSI+KMeans
# Подберите n_components под размер корпуса; 50 — безопасная отправная точка
lsi_model = LsiKMeansTopicModel(n_components=50, n_clusters=12, max_df=1.0, min_df=2, ngram_range=(1,2))
lsi_model.fit(texts)

# Топ-термины кластеров
lsi_terms = pd.DataFrame.from_dict(lsi_model.get_cluster_terms(texts, top_n=10), orient="index")
display(lsi_terms)

# Автоназвания кластеров из n-грамм (1–3)
kmeans_auto = generate_kmeans_labels(lsi_model, texts, final_top_k=1, ngram_range=(1,3), min_df=2)

# Таблица документов с кластером и ярлыком
df_lsi = lsi_model.to_dataframe(texts)
df_lsi["auto_label"] = df_lsi["topic_id"].map({cid: (labs[0] if labs else "") for cid, labs in kmeans_auto.items()})
display(df_lsi.head())

